In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import timm
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정 
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
seed_everything(5148)

In [3]:
dataset_path = './Rice_Image_Dataset'
classes = []

for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):  # Check if it's a directory
        classes.append(folder)


In [4]:
img_sample = []
for cat in classes:
    folder_path = os.path.join(dataset_path, cat)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = Image.open(img_path)
        img_sample.append(img)
        break

In [5]:
all_images = []
all_labels = []

for label in classes:
    folder = os.path.join(dataset_path, label)
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        all_images.append(img_path)
        all_labels.append(label)

In [6]:
# Split data into training and temporary (val+test) sets
train_images, temp_images, train_labels, temp_labels = train_test_split(
    all_images, all_labels, test_size=0.4, stratify=all_labels, random_state=42
)

# Split the temporary set into validation and test sets
val_images, test_images, val_labels, test_labels = train_test_split(
    temp_images, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42
)

In [7]:
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        
        self.label_to_int = {label: idx for idx, label in enumerate(sorted(set(labels)))}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')  # Convert to RGB in case some images are grayscale

        if self.transform:
            image = self.transform(image)
        
        # Convert the label from string to integer
        label = self.label_to_int[label]

        return image, label


In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
train_dataset = CustomImageDataset(train_images, train_labels, transform=transform)
val_dataset = CustomImageDataset(val_images, val_labels, transform=transform)
test_dataset = CustomImageDataset(test_images, test_labels, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [9]:
# VGGNet 사전 정의
class VGGNet(nn.Module):
    def __init__(self, num_classes=5): 
        super(VGGNet, self).__init__()
        self.vggnet= timm.create_model('vgg16', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        return self.vggnet(x)
    
# 모델 불러오기
vgg_model = VGGNet().to(device)
vgg_model.load_state_dict(torch.load('pre_VGGNet_seed.pt'))
# 평가모델로 변경
vgg_model.eval()

VGGNet(
  (vggnet): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): 

In [10]:
# ResNet 사전 정의
class ResNet(nn.Module):
    def __init__(self, num_classes=5):  # Assuming 5 classes for classification
        super(ResNet, self).__init__()
        self.resnet= timm.create_model('resnet50', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        return self.resnet(x)
    
# 모델 불러오기
resnet_model = ResNet().to(device)
resnet_model.load_state_dict(torch.load('pre_ResNet_seed.pt'))
# 평가모델로 변경
resnet_model.eval()

ResNet(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act2): ReLU(inplace=True)
        (aa): Identity()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, moment

In [11]:
# EffNet 사전 정의
class EffNet(nn.Module):
    def __init__(self, num_classes=5):  # Assuming 5 classes for classification
        super(EffNet, self).__init__()
        
        # Timm 라이브러리를 사용하여 efficientnet_b0 모델 불러오기
        self.efficientnet = timm.create_model('efficientnet_b0', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        return self.efficientnet(x)

# 모델 불러오기
effnet_model = EffNet().to(device)
effnet_model.load_state_dict(torch.load('pre_effNet_seed.pt'))
# 평가모델로 변경
effnet_model.eval()

EffNet(
  (efficientnet): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1

In [12]:
# DenseNet 사전 정의
class DenseNet(nn.Module):
    def __init__(self, num_classes=5):  # Assuming 5 classes for classification
        super(DenseNet, self).__init__()
        self.densenet= timm.create_model('densenet121', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        return self.densenet(x)
# 모델 불러오기
densenet_model = DenseNet().to(device)
densenet_model.load_state_dict(torch.load('pre_DenseNet_seed.pt'))
# 평가모델로 변경
densenet_model.eval()

DenseNet(
  (densenet): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNormAct2d(
        64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): ReLU(inplace=True)
      )
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): DenseBlock(
        (denselayer1): DenseLayer(
          (norm1): BatchNormAct2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNormAct2d(
            128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (conv2): Conv2d(128, 32, kerne

In [13]:
# NasNet 사전 정의
class NasNet(nn.Module):
    def __init__(self, num_classes=5):  # Assuming 5 classes for classification
        super(NasNet, self).__init__()
        self.nasnet= timm.create_model('mnasnet_100', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        return self.nasnet(x)
    
# 모델 불러오기
nasnet_model = NasNet().to(device)
nasnet_model.load_state_dict(torch.load('pre_NasNet_seed.pt'))
# 평가모델로 변경
nasnet_model.eval()

NasNet(
  (nasnet): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (se): Identity()
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNormAct2d(
            16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): Identity()
          )
          (drop_path): Id

In [14]:
# VIT 사전 정의
class VIT(nn.Module):
    def __init__(self, num_classes=5):  # Assuming 5 classes for classification
        super(VIT, self).__init__()
        self.vit= timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        return self.vit(x)

# 모델 불러오기
vit_model = VIT().to(device)
vit_model.load_state_dict(torch.load('pre_VIT_seed.pt'))
# 평가모델로 변경
vit_model.eval()

VIT(
  (vit): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          

In [15]:
correct = 0
total = 0


with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        vit_outputs = torch.nn.functional.softmax(vit_model(inputs), dim=1)
        resnet_outputs = torch.nn.functional.softmax(resnet_model(inputs), dim=1)
        nasnet_outputs = torch.nn.functional.softmax(nasnet_model(inputs), dim=1)
        vgg_outputs = torch.nn.functional.softmax(vgg_model(inputs), dim=1)
        effnet_outputs = torch.nn.functional.softmax(effnet_model(inputs), dim=1)
        densenet_outputs = torch.nn.functional.softmax(densenet_model(inputs), dim=1)
        # 각 모델의 확률 예측값을 평균
        soft_voting_probs = (vit_outputs+resnet_outputs+nasnet_outputs+vgg_outputs+effnet_outputs+densenet_outputs) / 6
        # 최종 예측값 선택
        ensemble_predictions = torch.argmax(soft_voting_probs,1)
        total += labels.size(0)
        correct += (ensemble_predictions == labels).sum().item()

accuracy = correct / total * 100
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 99.9133
